In [63]:
# Set team names
non_bibs_team_name = "Non-bibs FC"
bibs_team_name = "Bibs FC"

team_dict = {
    "n": non_bibs_team_name, "b": bibs_team_name
}

In [66]:
from moviepy.config import change_settings

change_settings({
    "IMAGEMAGICK_BINARY": r"C:\\Program Files\\ImageMagick-7.1.2-Q16-HDRI\\magick.exe"
})

def mm_ss_to_seconds(time_float):
    """
    Converts time from MM.SS format to total seconds.
    Example: 1.51 → 111 seconds (1 min + 51 sec)
    """
    minutes = int(time_float)
    seconds = int(round((time_float - minutes) * 100))
    return minutes * 60 + seconds


def match_tracker():
    score = {non_bibs_team_name: 0, bibs_team_name: 0}
    events = {non_bibs_team_name: [], bibs_team_name: []}
    
    team, scorer, assist = yield  # Prime the generator with initial input
    
    while True:
        if team not in score:
            raise ValueError(f"Team must be '{non_bibs_team_name}' or '{bibs_team_name}'")
        
        score[team] += 1
        events[team].append(f"{scorer} ({assist})")
        
        # Build output string
        output_lines = []
        for t in [non_bibs_team_name, bibs_team_name]:
            output_lines.append(f"{t}: {score[t]}")
            output_lines.extend(events[t])
            output_lines.append("")  # Blank line between teams
        
        output_text = "\n".join(output_lines).strip()
        
        # Wait for next input and yield output simultaneously
        team, scorer, assist = yield output_text

In [ ]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip

# === Configure your match video ===
video = VideoFileClip("..\July 15, 2025 Game 2 - Copy.mp4")

# === Sample goal timestamps (start, end, score shown) in seconds ===
highlights = [
    {"start": 450, "end": 480, "score": "1 - 0"},
    {"start": 1320, "end": 1350, "score": "2 - 1"},
    {"start": 2250, "end": 2280, "score": "3 - 2"},
]

highlights = [
    {"time":7.07, "team":"n", "scored":"Pavlos","assist":"Tom"},
    {"time":8.20, "team":"n", "scored":"Manzan","assist":"Chris"},
    {"time":10.06, "team":"n", "scored":"Tom","assist":"Alex D"},
    {"time":20.59, "team":"n", "scored":"Pavlos"}
]

# === Create clips with score overlay ===
highlight_clips = []
# Create the generator
tracker = match_tracker()
next(tracker) 
for i, h in enumerate(highlights):
    start = mm_ss_to_seconds(h["start"] if "start" in h else h["time"]) - 10
    end = mm_ss_to_seconds(h["end"] if "end" in h else h["time"]) + 5
    clip = video.subclip(start, end)
    
    # Create score text overlay
    text_duration = h["text_duration"] if "text_duration" in h else 5
    text = tracker.send((team_dict[h["team"]], h["scored"], h["assist"] if "assist" in h else ""))
    txt = TextClip(
        text, fontsize=48, color='white', font="Arial-Bold", bg_color='black'
    ).set_position(("center", "bottom")).set_duration(text_duration).set_start(clip.duration - text_duration)
    
    composite = CompositeVideoClip([clip, txt])
    highlight_clips.append(composite)

# === Concatenate all highlight clips ===
from moviepy.editor import concatenate_videoclips

final_highlights = concatenate_videoclips(highlight_clips)
final_highlights.write_videofile("match1_highlights_test.mp4", codec="libx264", fps=25)
 

Moviepy - Building video match1_highlights_test.mp4.
MoviePy - Writing audio in match1_highlights_testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video match1_highlights_test.mp4



Moviepy - Done !
Moviepy - video ready match1_highlights_test.mp4
